# Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

Now that we know (from the prior Notebook) how to query tabular data on a CSV file, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM so advanced as GPT-4 can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. However the same code below works with any SQL database like Synapse for example. The server should be created on the Resource Group where the Azure Cognitive Search service is located.

Let's begin..

In [1]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit # tool for interacting with SQL db
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv()

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

# Install MS SQL DB driver in your machine

We need the driver installed on this compute in order to talk to the SQL DB, so run the below cell once<br>
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline



In [7]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {
                'drivername': 'mssql+pyodbc',
                'username': os.environ["SQL_SERVER_USERNAME"] +'@'+ os.environ["SQL_SERVER_NAME"],
                'password': os.environ["SQL_SERVER_PASSWORD"],
                'host': os.environ["SQL_SERVER_NAME"],
                'port': 1433,
                'database': os.environ["SQL_SERVER_DATABASE"],
                'query': {'driver': 'ODBC Driver 17 for SQL Server'}
            }

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tNov  2 2023 01:40:17 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


In [11]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    #Creates the table in Azure SQL
    engine.execute(create_table_sql)
    print("Table",table_name,"succesfully created")
    # Insert data into SQL Database
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

(pyodbc.ProgrammingError) ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'covidtracking' in the database. (2714) (SQLExecDirectW)")
[SQL: CREATE TABLE covidtracking (date VARCHAR(MAX), state VARCHAR(MAX), death FLOAT, deathConfirmed FLOAT, deathIncrease INT, deathProbable FLOAT, hospitalized FLOAT, hospitalizedCumulative FLOAT, hospitalizedCurrently FLOAT, hospitalizedIncrease INT, inIcuCumulative FLOAT, inIcuCurrently FLOAT, negative FLOAT, negativeIncrease INT, negativeTestsAntibody FLOAT, negativeTestsPeopleAntibody FLOAT, negativeTestsViral FLOAT, onVentilatorCumulative FLOAT, onVentilatorCurrently FLOAT, positive FLOAT, positiveCasesViral FLOAT, positiveIncrease INT, positiveScore INT, positiveTestsAntibody FLOAT, positiveTestsAntigen FLOAT, positiveTestsPeopleAntibody FLOAT, positiveTestsPeopleAntigen FLOAT, positiveTestsViral FLOAT, recovered FLOAT, totalTestEncountersViral FLOAT, totalTestEncountersViralIncrease INT, to

# Query with LLM

**Note**: We are here using Azure SQL, however the same code will work with Synapse, SQL Managed instance, or any other SQL engine. You just need to provide the right ENV variables and it will connect succesfully.

In [12]:
# Create an LLM Langchain object using GPT-4 deployment
# Again we need GPT-4. It is necesary in the use of Agents. GPT-35-Turbo will make many mistakes.
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=1000)

In [13]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url)

In [14]:
# Natural Language question (query)
QUESTION = "How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states? Use the hospitalizedIncrease column"

### SQL Agent

Let's use an agent now and see how ReAct framework solves the problem.

In [15]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [16]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.agent.allowed_tools

['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

In [18]:
# And let's see now our clever crafted prompt
# printmd(agent_executor.agent.llm_chain.prompt.template)

In [19]:
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: covidtracking
Thought:The `covidtracking` table seems to be the most relevant one for this question. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "covidtracking"
Observation: 
CREATE TABLE covidtracking (
	date VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	state VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	death FLOAT(53) NULL, 
	[deathConfirmed] FLOAT(53) NULL, 
	[deathIncrease] INTEGER NULL, 
	[deathProbable] FLOAT(53) NULL, 
	hospitalized FLOAT(53) NULL, 
	[hospitalizedCumulative] FLOAT(53) NULL, 
	[hospitalizedCurrently] FLOAT(53) NULL, 
	[hospitalizedIncrease] INTEGER NULL, 
	[inIcuCumulative] FLOAT(53) NULL, 
	[inIcuCurrently] FLOAT(53) NULL, 
	negative FLOAT(53) NULL, 
	[negativeIncrease] INTEGER NULL, 
	[negativeTestsAntibody] FLOAT(53) NULL, 
	[negativeTestsPeopleAntibody] FLOAT(53) NULL

In [20]:
printmd(response)

In July 2020, there were 0 patients hospitalized in Texas and 63,105 patients hospitalized nationwide.

Explanation:
I queried the `covidtracking` table for the sum of the `hospitalizedIncrease` column where the state is 'TX' and the date starts with '2020-07'. The query returned 0, which means there were no patients hospitalized in Texas in July 2020. 

I used the following query

```sql
SELECT SUM(hospitalizedIncrease) FROM covidtracking WHERE state = 'TX' AND date LIKE '2020-07%'
```

Then, I queried the `covidtracking` table for the sum of the `hospitalizedIncrease` column where the date starts with '2020-07'. The query returned 63105, which means there were 63,105 patients hospitalized nationwide in July 2020.

I used the following query

```sql
SELECT SUM(hospitalizedIncrease) FROM covidtracking WHERE date LIKE '2020-07%'
```

Bad pipe message: %s [b'wh\x89\xeb\x1a\x98-(\xba\x8fV\x8a\x03X\x1b?\xe3W \xd8Z\xb14\xfd\xa6\xcb\xfd\xcb99\xfc?\xa4\xc5m\xdc\xdd\x049U\xd8\x16\xfa\x14~\x0ba\xc3\x80b\x07\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b']
Bad pipe message: %s [b'\x08\x05\x08\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xb7\x9d\x05x\xecIS?\xdf<Q?\xf9ui\xd0\x9a\x91eE\xa5\xb6']
Bad pipe message: %s [b'\xc9fr\x1cE9T1\xd4\x17\x18\xb0A\xb0\x03\xbd\xa0\xd3 \xc6{\xf4\xdb\xfb\xe6\xd3\x88S\xd5\xbfd\xe7r\x10rD\x151\x93\xcb\xe4\xb8\x9bFds\x9ao\xa9\x17\xe3\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00

**IMPORTANT NOTE**: If you don't specify the column name on the question, runing the above cell multiple times will yield diferent results some times. <br>
The reason is:
The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT-4.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. GPT-4 has been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will show you how to create a custom REACT agent that connects to the internet using BING SEARCH API to answer questions grounded on search results with citations. Basically a clone of Bing Chat.